**Ансамбль моделей** — это метод, в котором несколько алгоритмов (или вариации одного и того же) обучаются на одних данных, а итоговый прогноз строится на основе всех полученных от моделей прогнозов.

# <center>Бутстреппинг. Бэггинг</center>

В основе бэггинга лежит статистический метод, который называется **бутстрепом (bootstrap)**. Идея бутстрепа заключается в генерации выборок размера n из исходного датасета размера N путём случайного выбора элементов с повторениями в каждом из наблюдений.

![image.png](https://lms.skillfactory.ru/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block@MATHML_md9_2_1.png)

In [28]:
# Задание 2.2

import numpy as np

a = np.array([4.4, 3.8, 4.8, 4.0, 3.4, 4.2, 5.2])
b = a.mean()
round(np.sum((a - b)**2)/7, 3)

0.317

## <center>Bias и Variance</center>

**Смещение** — это разница между математическим ожиданием для прогноза и реальным значением:

$$Bias[\hat{f}(x)] = E[\hat{f}(x)]-y$$

> Смысл смещения — способность получить лучшую среди всех возможных моделей, то есть максимально точные прогнозы.

**Алгоритмы со стабильно маленьким смещением:**

* KNN (n=1)
* метод опорных векторов
* деревья решений с большой глубиной

**Алгоритмы с большим смещением:**

* линейная регрессия
* логистическая регрессия
* деревья решений с маленькой глубиной

**Разброс** — это величина разницы в результатах обучения модели на разных выборках:

$$\operatorname{Var}[\hat{f}(x)]=\mathrm{E}\left[\left(\mathrm{E}[\hat{f}(x)]-\hat{f}(x)\right)^{2}\right]$$

Разброс характеризует устойчивость модели к изменениям в обучающей выборке:

* Если результат сильно зависит от того, какие объекты присутствуют в выборке, разброс будет большим.

* Если алгоритм работает стабильно вне зависимости от особенностей выборки, разброс будет маленьким.

**Алгоритмы, споказывающие маленький разброс:**

* линейная регрессия
* логистическая регрессия
* деревья решений с маленькой глубиной

**Алгоритмы, споказывающие большой разброс:**

* деревья решений с большой глубиной
* KNN
* метод опорных векторов

> Наша ошибка — это сумма квадрата смещения, разброса и неустранимой случайной ошибки. Теперь мы понимаем, из чего состоит ошибка модели. Такое представление помогает нам исследовать с теоретической точки зрения некоторые алгоритмы машинного обучения и часто используется при изучении ансамблей.

## <center>Бэггинг</center>

Его идея состоит в том, что мы берём несколько независимых моделей и усредняем полученные по ним результаты. Таким образом мы получаем модель, имеющую меньший разброс, так как при её построении мы учли несколько моделей.

Как уже было сказано, в реальности получить много независимых выборок слишком сложно, так как найти столько данных обычно невозможно. Поэтому мы используем бутстреп-выборки.

> Важно отметить, что при бэггинге размер каждой бутстреп-выборки должен совпадать с размером исходной выборки.

* Если мы рассматриваем задачу классификации, то, по сути, модели «голосуют» за свой класс.
* Если мы рассматриваем задачу регрессии, то результат — просто среднее арифметическое прогнозов по всем моделям.

**Резюмируем**:

* Бэггинг даёт уменьшение ошибки в  раз по сравнению с одиночной моделью.
* Бэггинг не уменьшает смещение по сравнению с одиночной моделью.
* Бэггинг уменьшает разброс в  раз по сравнению с одиночной моделью.

> Важно отметить, что эти утверждения выведены и доказаны теоретически и будут выполняться на практике только в том случае, если между ошибками нулевая корреляция.

In [29]:
# Задание 2.7

import pandas as pd
import numpy as np

In [30]:
df = pd.read_csv('data/wineQualityReds.csv')
df.head()

,Unnamed: 0,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
0,1,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,2,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,3,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,4,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,5,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [31]:
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

In [32]:
df['target'] = np.where(df.quality >=6,1,0)

y= df['target']
X = df.drop(['target', 'quality'], axis= 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [33]:
model = LogisticRegression()
model.fit(X_train, y_train)
preds_train = model.predict(X_train)
preds_test = model.predict(X_test)
f1_score(preds_test, y_test)

c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7390476190476191

In [34]:
model = DecisionTreeClassifier(max_depth = 10, random_state=42)
model.fit(X_train, y_train)
preds_train = model.predict(X_train)
preds_test = model.predict(X_test)
f1_score(preds_test, y_test)

0.7601476014760148

In [35]:
# Задание 2.8

from sklearn.ensemble import BaggingClassifier

model = BaggingClassifier(estimator=DecisionTreeClassifier(max_depth = 10, random_state=42), n_estimators=1500, random_state=42).fit(X_train, y_train)
preds_train = model.predict(X_train)
preds_test = model.predict(X_test)
f1_score(preds_test, y_test)

0.8239700374531835

## <center>Случайный лес</center>

![image.pmg](https://lms-cdn.skillfactory.ru/assets/courseware/v1/5447a7937de6636597fac98b4f0a9f62/asset-v1:Skillfactory+DSMED+2023+type@asset+block/MATHML_md9_3_6.png)

> Одно из важных понятий, которое здесь появляется, — это **метод случайных подпространств**, который используется для построения ансамблей моделей.

Кратко опишем его принцип:

1) Отбираем обучающую выборку.
2) Определяем число моделей, которые войдут в ансамбль.
3) Для каждой модели берём не все признаки, а только часть из них и формируем выборку с использованием случайно выбранного набора признаков.
4) Объединяем все результаты и определяем итоговое решение по объектам.

Алгоритм случайного леса в таком контексте реализуется следующим образом:

1) Для того чтобы построить $i$-е дерево леса, из обучающей выборки $X$ берём случайную подвыборку $X_i$ того же размера, что и вся обучающая выборка.

2) После этого в каждой вершине каждого дерева из $M$ возможных признаков выбираем случайную группу признаков объёма $L$. Для выбранных признаков ищем оптимальное разбиение. Рекомендуется использовать $L=\sqrt{M}$ в задачах классификации и $\frac{M}{3}$ — в задачах регрессии.

3) Для получения предсказания необходимо воспользоваться обычным принципом бэггинга: взять усреднённый ответ в случае регрессии или самый популярный класс — для классификации.

**Ошибка Out-of-Bag** — это способ оценить качество случайного леса.

Для того чтобы найти *out-of-Bag*-оценку:

1) Для каждого объекта $x_i$ получаем предсказания всех деревьев $a_b$, обучавшихся на бутстреп-выборках $X_b$, не содержащих $x_i$.

2) Усредняем эти предсказания.

3) Находим значение ошибки для усреднённого предсказания.

4) Усредняем значение функционала ошибки для всех объектов выборки.

In [36]:
# Задание 3.4

data = pd.read_csv('data/boston (1).csv')
data.head()

,crim_rate,zn,business,river,nit_oxiden,rooms,age,dist,highways_index,tax,pup_per_teaс,lower,target
0,"0,00632",18,"2,31",0,"0,538","6,575","65,2","4,09",1,296,"15,3","4,98",24
1,"0,02731",0,"7,07",0,"0,469","6,421","78,9","4,9671",2,242,"17,8","9,14","21,6"
2,"0,02729",0,"7,07",0,"0,469","7,185","61,1","4,9671",2,242,"17,8","4,03","34,7"
3,"0,03237",0,"2,18",0,"0,458","6,998","45,8","6,0622",3,222,"18,7","2,94","33,4"
4,"0,06905",0,"2,18",0,"0,458","7,147","54,2","6,0622",3,222,"18,7","5,33","36,2"


In [37]:
# # 1.

# import statistics

# X = data.drop('target', axis=1)
# y = data['target']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 13)
# y_train.mean()

In [38]:
# # 2.

# from sklearn.linear_model import LinearRegression 
# from sklearn.metrics import mean_absolute_error as mae

# model_lr = LinearRegression()
# model_lr.fit(X_train, y_train)
# preds_train = model_lr.predict(X_train)
# preds_test = model_lr.predict(X_test)
# print(mae(y_test, preds_test))

In [39]:
# # 3.

# from sklearn.tree import DecisionTreeClassifier
# from sklearn.metrics import mean_absolute_error as mae

# model = DecisionTreeRegressor(random_state=13)
# model.fit(X_train, Y_train)
# preds_train = model.predict(X_train)
# preds_test = model.predict(X_test)
# print(mean_absolute_error(y_train, preds_train))
# print(mean_absolute_error(y_test, preds_test))

In [40]:
# # 4.

# from sklearn.ensemble import RandomForestClassifier

# for n in [3, 10, 100, 500]:
#     model = RandomForestRegressor(n_estimators=n, random_state = 13)
#     model.fit(X_train, y_train)
#     preds_train = model.predict(X_train)
#     preds_test = model.predict(X_test)
#     print(mean_absolute_error(y_test, preds_test))

In [41]:
df = pd.read_csv('data/weatherAUS.csv')
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [42]:
# Задание 4.1

df.isnull().sum().sum()

343248

In [43]:
# Задание 4.2

round(df.isna().sum() / len(df), 3)

Date             0.000
Location         0.000
MinTemp          0.010
MaxTemp          0.009
Rainfall         0.022
Evaporation      0.432
Sunshine         0.480
WindGustDir      0.071
WindGustSpeed    0.071
WindDir9am       0.073
WindDir3pm       0.029
WindSpeed9am     0.012
WindSpeed3pm     0.021
Humidity9am      0.018
Humidity3pm      0.031
Pressure9am      0.104
Pressure3pm      0.103
Cloud9am         0.384
Cloud3pm         0.408
Temp9am          0.012
Temp3pm          0.025
RainToday        0.022
RainTomorrow     0.022
dtype: float64

In [44]:
df.drop(['Evaporation','Sunshine','Cloud3pm'], axis = 1, inplace = True)

In [45]:
# Задание 4.3

df.RainToday = df.RainToday.map({'No': 0, 'Yes': 1})

In [46]:
df.RainTomorrow = df.RainTomorrow.map({'No': 0, 'Yes': 1})

In [47]:
# Задание 4.4

df.Date = pd.to_datetime(df.Date)
df['Month'] = df.Date.dt.month
df.drop('Date', axis = 1, inplace = True)
df_season = df.groupby('Month').mean(numeric_only=True)
df_season[['RainToday']]

,RainToday
Month,
1,0.189484
2,0.206746
3,0.217135
4,0.216845
5,0.222163
6,0.263638
7,0.270736
8,0.253167
9,0.229135


In [48]:
# Задание 4.5

categoricals = ['Month', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm']
df_dummies = pd.get_dummies(df, columns=categoricals)
df_dummies.shape

(145460, 124)

In [49]:
# Задание 4.6

df_dummies.dropna(inplace=True)
X = df_dummies.drop('RainTomorrow', axis = 1)
y = df_dummies['RainTomorrow']  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 31)
y_test.mean()

0.22770253002811142

In [50]:
# Задание 4.7

def gbs(data, n):     
    inds = np.random.randint(0, len(data), (n, len(data))) #определяем индексы случайным образом
    numbers = data[inds] #выбираем значения по индексам
    return numbers

In [51]:
target = X_train['MinTemp'].values #выбираем целевую переменную
np.random.seed(31) #задаём параметр генератора случайных чисел
mean_values = [np.mean(x) for x in gbs(target, 1000)] #получаем все средние значения
np.std(mean_values) #находим для них стандартное отклонение

0.02879072820657669

In [52]:
# Задание 4.8

from sklearn.metrics import roc_auc_score

clf = LogisticRegression()
clf.fit(X_train, y_train)
preds_train = clf.predict(X_train)
preds_test = clf.predict(X_test)
roc_auc_score(y_test, preds_test)

c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7265516156243416

In [53]:
# Задание 4.9

params = {
    'max_leaf_nodes': list(range(2, 10)), 
    'min_samples_split': [2, 3, 4], 
    'max_depth': [5,7,9,11]
}

In [54]:
from sklearn.model_selection import GridSearchCV

grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, verbose=3, cv=3)
grid_search_cv.fit(X_train, y_train)
print(grid_search_cv.best_params_)

Fitting 3 folds for each of 96 candidates, totalling 288 fits
[CV 1/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=2;, score=0.817 total time=   0.0s
[CV 2/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=2;, score=0.820 total time=   0.0s
[CV 3/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=2;, score=0.825 total time=   0.0s
[CV 1/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=3;, score=0.817 total time=   0.0s
[CV 2/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=3;, score=0.820 total time=   0.0s
[CV 3/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=3;, score=0.825 total time=   0.0s
[CV 1/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=4;, score=0.817 total time=   0.0s
[CV 2/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=4;, score=0.820 total time=   0.0s
[CV 3/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=4;, score=0.825 total time=   0.0s
[CV 1/3] END max_depth=5, max_leaf_nodes=3, min_samples_split=2;,

In [55]:
clf = DecisionTreeClassifier(max_depth = 5, max_leaf_nodes = 9, min_samples_split = 2, random_state=42)
clf.fit(X_train, y_train)
preds_train = clf.predict(X_train)
preds_test = clf.predict(X_test)
print(round(roc_auc_score(y_test, preds_test), 2))

0.7


In [56]:
# Задание 4.10

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators = 100, random_state=31)
clf.fit(X_train, y_train)
preds_train = clf.predict(X_train)
preds_test = clf.predict(X_test)
print(round(roc_auc_score(y_test, preds_test), 2))

0.73


In [57]:
# Задание 4.11

params = {
    'max_features': [4, 5, 6, 7], 
    'min_samples_leaf': [3, 5, 7, 9, 11], 
    'max_depth': [5, 10, 15]
}

In [58]:
grid_search_cv = GridSearchCV(RandomForestClassifier(random_state=31), params, verbose=3, cv=3)
grid_search_cv.fit(X_train, y_train)
print(grid_search_cv.best_params_)

Fitting 3 folds for each of 60 candidates, totalling 180 fits
[CV 1/3] END max_depth=5, max_features=4, min_samples_leaf=3;, score=0.776 total time=   0.7s
[CV 2/3] END max_depth=5, max_features=4, min_samples_leaf=3;, score=0.775 total time=   0.7s
[CV 3/3] END max_depth=5, max_features=4, min_samples_leaf=3;, score=0.775 total time=   0.7s
[CV 1/3] END max_depth=5, max_features=4, min_samples_leaf=5;, score=0.776 total time=   0.7s
[CV 2/3] END max_depth=5, max_features=4, min_samples_leaf=5;, score=0.775 total time=   0.7s
[CV 3/3] END max_depth=5, max_features=4, min_samples_leaf=5;, score=0.775 total time=   0.7s
[CV 1/3] END max_depth=5, max_features=4, min_samples_leaf=7;, score=0.776 total time=   0.7s
[CV 2/3] END max_depth=5, max_features=4, min_samples_leaf=7;, score=0.775 total time=   0.7s
[CV 3/3] END max_depth=5, max_features=4, min_samples_leaf=7;, score=0.775 total time=   0.7s
[CV 1/3] END max_depth=5, max_features=4, min_samples_leaf=9;, score=0.776 total time=   0.6

In [59]:
clf = RandomForestClassifier(n_estimators = 100, max_depth=15, max_features=7, min_samples_leaf=3, random_state=31)
clf.fit(X_train, y_train)
preds_train = clf.predict(X_train)
preds_test = clf.predict(X_test)
print(round(roc_auc_score(y_test, preds_test), 2))

0.7


In [60]:
# Задание 4.12

feature_names = [x for x in df_dummies if x != 'RainTomorrow']
pd.DataFrame({'feat': feature_names,
              'coef': clf.feature_importances_}).sort_values(by='coef', ascending=False)

,feat,coef
7,Humidity3pm,0.250783
2,Rainfall,0.079757
6,Humidity9am,0.070403
10,Cloud9am,0.067092
9,Pressure3pm,0.065272
...,...,...
50,Location_Newcastle,0.000000
62,Location_SalmonGums,0.000000
51,Location_Nhil,0.000000
52,Location_NorahHead,0.000000


# <center>Бустинг</center>

![image.png](https://lms-cdn.skillfactory.ru/assets/courseware/v1/86ec44dadb1105d1e0717764b7e9d2a9/asset-v1:Skillfactory+DSMED+2023+type@asset+block/MATHML_md9_5_1.png)

* В **бэггинге** все модели обучаются одновременно, независимо и параллельно. В качестве итогового предсказания берётся усреднённый ответ (в задаче регрессии) или делается прогноз по большинству голосов (в задаче классификации).

* В **бустинге** все модели обучаются поочерёдно, причём каждая последующая старается исправить ошибки, совершённые предыдущими.

## <center>AdaBoost (Adaptive Boosting)</center>

**Бустинг** позволяет из большого количества относительно слабых и простых моделей получить одну сильную. В нашем случае будут рассматриваться деревья решений ограниченной глубины (всего из одного уровня) — их ещё называют **пнями**.

Пропишем сам алгоритм:

Инициализируем веса объектов:

$$w_{j}=\frac{1}{N}, j=1,2, \ldots, N$$

Для всех $i$ от 1 до $K$ (если у нас $K$ базовых моделей):

2.1. Строим классификатор $a_i(x)$, используя веса $w_j$.

2.2. Вычисляем ошибку:

$$\operatorname{err}_{i}=\sum_{j=1}^{N} w_{j}\left[y_{j} \neq a_{i}\left(x_{j}\right)\right]$$

2.3. Вычисляем вес нового алгоритма:

$$c_{i}=\frac{1}{2} \ln \frac{1-e r r_{i}}{e r r_{i}}$$

2.4. Получаем новые веса объектов (классы определяются как -1 и +1):

$$w_{j} \leftarrow w_{j} \cdot \exp \left(c_{i}\left[y_{j} \neq a_{i}\left(x_{j}\right)\right]\right), j=1, \ldots, N$$

2.5. Нормируем веса объектов:

$$w_{j} \longleftarrow \frac{w_{j}}{\sum_{j=1}^{N} w_{j}}$$

Группируем полученные модели:

$$f_{K}(x)=\operatorname{sign}\left[\sum_{i=1}^{K} c_{i} a_{i}(x)\right]$$

> **$sign$** — это функция знака, которая извлекает знак действительного числа. Определяется следующим образом:

$$\operatorname{sign}(x)=\left\{\begin{aligned} 1, & x>0 \\ 0, & x=0 \\-1, & x<0 \end{aligned}\right.$$

> Объект, который был классифицирован неправильно, имеет вес, более чем в два раза превышающий вес других объектов. Это означает, что он с большей вероятностью будет выбран несколько раз, и, таким образом, следующий пень будет больше сосредоточен на правильной классификации неправильно классифицированного образца. В этом и состоит идея *AdaBoost*.

## <center>Градиентный бустинг</center>

> Принцип его работы аналогичен *AdaBoost*: следующие модели улучшают композицию построенных ранее.

Инициализируем композицию *GBM (Gradient Boosting Machine)* — $f(x) = a_0 (x)$, то есть добавляем первый базовый алгоритм. Например, можно использовать:

* алгоритм $a_0 (x) = 0$, который всегда возвращает 0 (в задаче регрессии);

* более сложный алгоритм $a_0 (x) = \frac{1}{N} \sum_{j=1}^N y_i$, который возвращает средний истинный ответ по всем элементам обучающей выборки (в задаче регрессии);

* алгоритм $a_0 (x) = arg \ max_{y \in Y} \sum_{j=1}^N \left [y_i = y  \right ]$, который всегда возвращает метку самого распространённого класса в обучающей выборке (в задаче классификации).

Итеративно повторяем следующие три шага:

2.1. Вычисляем вектор сдвига:

$$s=-\nabla F=\left[\begin{array}{c}-L_{\hat{y}}^{\prime}\left(y_{1}, a_{K-1}\left(x_{1}\right)\right) \\ -L_{\hat{y}}^{\prime}\left(y_{2}, a_{K-1}\left(x_{2}\right)\right) \\ \vdots \\ -L_{\hat{y}}^{\prime}\left(y_{N}, a_{K-1}\left(x_{N}\right)\right)\end{array}\right]$$

2.2. Строим очередной базовый алгоритм $a_K (x)$, который предсказывает вектор-сдвиг:

$$a_{K}(x)=\arg \min _{a} \frac{1}{N} \sum_{j=1}^{N}\left(a\left(x_{j}\right)-s_{j}\right)^{2}$$

2.3. Добавляем $a_K (x)$ в композицию:

$$a_{K}(x)=\sum_{i=1}^{K} a_{i}(x)$$

Если выполнен критерий остановки, останавливаем итеративный процесс.

## <center>XGBoost и CatBoost</center>

**XGBoost** — одна из самых эффективных реализаций алгоритма *Gradient Boosted Trees*. Название *XGBoost* расшифровывается  как eXtreme Gradient Boosting. *XGBoost* — это улучшение *GBM* через системную оптимизацию и усовершенствование алгоритма.

**CatBoost** — это библиотека градиентного бустинга, созданная Яндексом. Её особенность заключается в том, что в ней используются так называемые небрежные (oblivious) деревья решений, чтобы «вырастить» сбалансированное дерево.

> Одно из **главных преимуществ** *CatBoost* заключается в том, что его можно использовать для данных, где категориальные признаки заранее не были преобразованы.

In [61]:
df = pd.read_csv('data/AirPass.csv')
df.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


In [64]:
df = df.drop('Unnamed: 0', axis = 1)
df.head()

,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


In [65]:
# Задание 6.1

df.isnull().sum().sum()

310

In [67]:
# Задание 6.2

df['Arrival Delay in Minutes'] = df['Arrival Delay in Minutes'].fillna(df['Arrival Delay in Minutes'].median())
round(df['Arrival Delay in Minutes'].mean(), 2)

15.13

In [68]:
# Задание 6.3

# 1.

df.groupby('Gender')['satisfaction'].value_counts()*100/df.shape[0]

Gender  satisfaction           
Female  neutral or dissatisfied    29.058554
        satisfied                  21.687327
Male    neutral or dissatisfied    27.608177
        satisfied                  21.645942
Name: count, dtype: float64

In [69]:
# 2.

df.groupby('Type of Travel')['satisfaction'].value_counts()*100/df.shape[0]

Type of Travel   satisfaction           
Business travel  satisfied                  40.177472
                 neutral or dissatisfied    28.785225
Personal Travel  neutral or dissatisfied    27.881506
                 satisfied                   3.155798
Name: count, dtype: float64

In [70]:
# 3.

df.groupby('Class')['satisfaction'].value_counts()*100/df.shape[0]

Class     satisfaction           
Business  satisfied                  33.184478
          neutral or dissatisfied    14.614452
Eco       neutral or dissatisfied    36.614567
          satisfied                   8.374076
Eco Plus  neutral or dissatisfied     5.437712
          satisfied                   1.774715
Name: count, dtype: float64

In [71]:
# Перекодируем часть бинарных признаков, чтобы использовать их при обучении
df['satisfaction'] = df['satisfaction'].map({'neutral or dissatisfied':0 , 'satisfied':1})
df['Customer Type'] = df['Customer Type'].map({'Loyal Customer':1, 'disloyal Customer':0})
df['Type of Travel'] = df['Type of Travel'].map({'Personal Travel':0, 'Business travel':1})
df['Gender'] = df['Gender'].map({'Male': 0, 'Female': 1})

In [72]:
# Задание 6.4

df=pd.get_dummies(df)
df.shape

(103904, 26)

In [76]:
# Задание 6.5

X = df.drop('satisfaction', axis = 1)
y = df['satisfaction']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=26)
y_test.shape

(20781,)

In [77]:
# Задание 6.6

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train) 
X_train = scaler.transform(X_train) 
X_test = scaler.transform(X_test) 
X_test[0][0]

0.9408251379303

In [78]:
# Задание 6.7

model_lr = LogisticRegression()
model_lr.fit(X_train, y_train)
preds_test = model_lr.predict(X_test)
f1_score(preds_test, y_test)

0.8547046934650797

In [79]:
# Задание 6.8

from sklearn.ensemble import AdaBoostClassifier

model_ada = AdaBoostClassifier(DecisionTreeClassifier(random_state=26),random_state=26,learning_rate=0.01)

model_ada.fit(X_train, y_train)
preds_test = model_ada.predict(X_test)
f1_score(preds_test, y_test)

c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


0.9404794558121674

In [82]:
import warnings
warnings.filterwarnings('ignore')

In [86]:
# Задание 6.9

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import make_scorer

params = {"n_estimators":2**np.arange(8), "learning_rate":0.1**np.arange(3)}

model_for_gs = GradientBoostingClassifier()
params = {"n_estimators":2**np.arange(8), "learning_rate":0.1**np.arange(3)}
gs = GridSearchCV(model_for_gs, 
                  params, 
                  cv=3, 
                  scoring=make_scorer(f1_score),
                  verbose=5)
 
gs.fit(X_train, y_train)
 
print("Лучшие гиперпараметры:", gs.best_params_)
print("Лучшее значение метрики:", gs.best_score_)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV 1/3] END .learning_rate=1.0, n_estimators=1;, score=0.873 total time=   0.0s
[CV 2/3] END .learning_rate=1.0, n_estimators=1;, score=0.870 total time=   0.0s
[CV 3/3] END .learning_rate=1.0, n_estimators=1;, score=0.871 total time=   0.0s
[CV 1/3] END .learning_rate=1.0, n_estimators=2;, score=0.880 total time=   0.1s
[CV 2/3] END .learning_rate=1.0, n_estimators=2;, score=0.878 total time=   0.1s
[CV 3/3] END .learning_rate=1.0, n_estimators=2;, score=0.875 total time=   0.1s
[CV 1/3] END .learning_rate=1.0, n_estimators=4;, score=0.901 total time=   0.3s
[CV 2/3] END .learning_rate=1.0, n_estimators=4;, score=0.896 total time=   0.3s
[CV 3/3] END .learning_rate=1.0, n_estimators=4;, score=0.897 total time=   0.3s
[CV 1/3] END .learning_rate=1.0, n_estimators=8;, score=0.920 total time=   0.6s
[CV 2/3] END .learning_rate=1.0, n_estimators=8;, score=0.920 total time=   0.7s
[CV 3/3] END .learning_rate=1.0, n_estimators=8;

In [87]:
# Задание 6.10

from xgboost import XGBClassifier

model_xgb = XGBClassifier(random_state=26)
model_xgb.fit(X_train,y_train)
preds_test = model_xgb.predict(X_test)
f1_score(preds_test, y_test)

0.9573723114544546

In [88]:
# Задание 6.11

from catboost import CatBoostClassifier

model = CatBoostClassifier(random_state=26)
model.fit(X_train, y_train)
preds_class = model.predict(X_test)
f1_score(preds_class, y_test)

Learning rate set to 0.068023
0:	learn: 0.6018151	total: 108ms	remaining: 1m 48s
1:	learn: 0.5020758	total: 116ms	remaining: 58s
2:	learn: 0.4472471	total: 125ms	remaining: 41.5s
3:	learn: 0.4028689	total: 133ms	remaining: 33.2s
4:	learn: 0.3674734	total: 142ms	remaining: 28.2s
5:	learn: 0.3397847	total: 150ms	remaining: 24.8s
6:	learn: 0.3121211	total: 158ms	remaining: 22.3s
7:	learn: 0.2917500	total: 166ms	remaining: 20.6s
8:	learn: 0.2749040	total: 174ms	remaining: 19.2s
9:	learn: 0.2575190	total: 182ms	remaining: 18s
10:	learn: 0.2473691	total: 190ms	remaining: 17.1s
11:	learn: 0.2377533	total: 198ms	remaining: 16.3s
12:	learn: 0.2279311	total: 206ms	remaining: 15.7s
13:	learn: 0.2212511	total: 215ms	remaining: 15.2s
14:	learn: 0.2100357	total: 225ms	remaining: 14.7s
15:	learn: 0.2025732	total: 234ms	remaining: 14.4s
16:	learn: 0.1942300	total: 242ms	remaining: 14s
17:	learn: 0.1877937	total: 251ms	remaining: 13.7s
18:	learn: 0.1832380	total: 259ms	remaining: 13.4s
19:	learn: 0.179

0.9604136550650506

In [89]:
# Задание 6.12

from catboost import Pool
from catboost.utils import get_confusion_matrix

cm = get_confusion_matrix(model, Pool(X_train, y_train))

In [90]:
cm

array([[46647.,   559.],
       [ 1273., 34644.]])

In [92]:
# Задание 6.13

pd.DataFrame(
    {
        "feature_importance": model.get_feature_importance(),
        "feature_names": df.drop(columns="satisfaction").columns,
    }
).sort_values(by=["feature_importance"], ascending=False)

,feature_importance,feature_names
6,26.001691,Inflight wifi service
4,17.854438,Type of Travel
11,7.582693,Online boarding
2,7.291082,Customer Type
22,5.334009,Class_Business
17,3.935882,Checkin service
16,3.675108,Baggage handling
3,3.653960,Age
9,3.537978,Gate location
18,2.840194,Inflight service


# <center>Стекинг</center>

**Стекинг** —  это агрегация ответов моделей машинного обучения. Подход использует понятие **базовых моделей**, каждая из которых обучается независимо от остальных, и **метамодели**, которая использует предсказания базовых моделей как признаки.

> Важно отметить, что **базовые алгоритмы и метамодель должны обучаться на разных выборках**, чтобы избежать переобучения.

Это можно делать с помощью **блендинга** или **стекинга**.

**Блендинг** является простейшей реализацией стекинга.

Предположим, у нас есть обучающая выборка $X$, которую мы делим пополам: первая часть используется для обучения базовых моделей, а на второй базовые модели делают предсказания — метапризнаки, на которых и обучается в дальнейшем метамодель.

![image.png](https://lms-cdn.skillfactory.ru/assets/courseware/v1/2d3283bba09d12ce993a7984cb111036/asset-v1:Skillfactory+DSMED+2023+type@asset+block/MATHML_md9_7_2.png)

К сожалению, у блендинга есть проблема: ни базовые модели, ни метамодель не обучаются на полных данных.

Эту проблему решает **стекинг**.

Чтобы в итоге все модели могли «познакомиться» с полным набором данных, можно использовать подход, аналогичный кросс-валидации: мы можем разделять выборку на $L$ частей, обучать модель на части $L-1$ и делать предсказание на оставшейся. Определённого правила для выбора количества частей нет, но, разумеется, чем больше их будет, тем выше будет качество (времени на обучение также будет потрачено больше).

![image.png](https://lms-cdn.skillfactory.ru/assets/courseware/v1/fa4292a928728baaf9f760df93701a99/asset-v1:Skillfactory+DSMED+2023+type@asset+block/MATHML_md9_7_3.png)

Также метод стекинга можно реализовать с более чем двумя уровнями — это называется **многоуровневым стекингом**.

In [117]:
df = pd.read_csv('data/Frogs_MFCCs.csv')
df.head()

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family,Genus,Species,RecordID
0,1.0,0.152936,-0.105586,0.200722,0.317201,0.260764,0.100945,-0.150063,-0.171128,0.124676,...,-0.108351,-0.077623,-0.009568,0.057684,0.118680,0.014038,Leptodactylidae,Adenomera,AdenomeraAndre,1
1,1.0,0.171534,-0.098975,0.268425,0.338672,0.268353,0.060835,-0.222475,-0.207693,0.170883,...,-0.090974,-0.056510,-0.035303,0.020140,0.082263,0.029056,Leptodactylidae,Adenomera,AdenomeraAndre,1
2,1.0,0.152317,-0.082973,0.287128,0.276014,0.189867,0.008714,-0.242234,-0.219153,0.232538,...,-0.050691,-0.023590,-0.066722,-0.025083,0.099108,0.077162,Leptodactylidae,Adenomera,AdenomeraAndre,1
3,1.0,0.224392,0.118985,0.329432,0.372088,0.361005,0.015501,-0.194347,-0.098181,0.270375,...,-0.136009,-0.177037,-0.130498,-0.054766,-0.018691,0.023954,Leptodactylidae,Adenomera,AdenomeraAndre,1
4,1.0,0.087817,-0.068345,0.306967,0.330923,0.249144,0.006884,-0.265423,-0.172700,0.266434,...,-0.048885,-0.053074,-0.088550,-0.031346,0.108610,0.079244,Leptodactylidae,Adenomera,AdenomeraAndre,1


In [118]:
df['Family'].value_counts()

Family
Leptodactylidae    4420
Hylidae            2165
Dendrobatidae       542
Bufonidae            68
Name: count, dtype: int64

In [119]:
df['Family'] = df['Family'].map({'Dendrobatidae':1, 'Leptodactylidae':0, 'Hylidae':0, 'Bufonidae':0})

In [120]:
df['Family'].value_counts()

Family
0    6653
1     542
Name: count, dtype: int64

In [121]:
df_final = df.drop(columns=['Genus', 'Species', 'RecordID'])

In [122]:
df_final

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_14,MFCCs_15,MFCCs_16,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family
0,1.0,0.152936,-0.105586,0.200722,0.317201,0.260764,0.100945,-0.150063,-0.171128,0.124676,...,0.082245,0.135752,-0.024017,-0.108351,-0.077623,-0.009568,0.057684,0.118680,0.014038,0
1,1.0,0.171534,-0.098975,0.268425,0.338672,0.268353,0.060835,-0.222475,-0.207693,0.170883,...,0.022786,0.163320,0.012022,-0.090974,-0.056510,-0.035303,0.020140,0.082263,0.029056,0
2,1.0,0.152317,-0.082973,0.287128,0.276014,0.189867,0.008714,-0.242234,-0.219153,0.232538,...,0.050791,0.207338,0.083536,-0.050691,-0.023590,-0.066722,-0.025083,0.099108,0.077162,0
3,1.0,0.224392,0.118985,0.329432,0.372088,0.361005,0.015501,-0.194347,-0.098181,0.270375,...,-0.011567,0.100413,-0.050224,-0.136009,-0.177037,-0.130498,-0.054766,-0.018691,0.023954,0
4,1.0,0.087817,-0.068345,0.306967,0.330923,0.249144,0.006884,-0.265423,-0.172700,0.266434,...,0.037439,0.219153,0.062837,-0.048885,-0.053074,-0.088550,-0.031346,0.108610,0.079244,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7190,1.0,-0.554504,-0.337717,0.035533,0.034511,0.443451,0.093889,-0.100753,0.037087,0.081075,...,-0.059364,0.024206,-0.000861,0.069430,0.071001,0.021591,0.052449,-0.021860,-0.079860,0
7191,1.0,-0.517273,-0.370574,0.030673,0.068097,0.402890,0.096628,-0.116460,0.063727,0.089034,...,-0.105600,0.030767,0.006457,0.061127,0.068978,0.017745,0.046461,-0.015418,-0.101892,0
7192,1.0,-0.582557,-0.343237,0.029468,0.064179,0.385596,0.114905,-0.103317,0.070370,0.081317,...,-0.078615,0.024861,0.008696,0.082474,0.077771,-0.009688,0.027834,-0.000531,-0.080425,0
7193,1.0,-0.519497,-0.307553,-0.004922,0.072865,0.377131,0.086866,-0.115799,0.056979,0.089316,...,-0.075320,0.022903,0.001924,0.051796,0.069073,0.017963,0.041803,-0.027911,-0.096895,0


In [123]:
X = df_final.drop('Family', axis = 1)
y = df_final['Family']

In [124]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=31)

In [125]:
# Задание 7.5

model_rfc = RandomForestClassifier(10, random_state=42)
model_rfc.fit(X_train, y_train)
pred = model_rfc.predict(X_test)
f1_score(y_test, pred)

0.9726775956284153

In [126]:
# Задание 7.6

from sklearn.neighbors import KNeighborsClassifier 
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import StackingClassifier

estimators = [('rf', RandomForestClassifier(n_estimators=10, random_state=31)),
              ('knn', KNeighborsClassifier(n_neighbors= 11)),
              ('nb', GaussianNB() )               
]
metamodel = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
)
metamodel.fit(X_train, y_train)
pred = metamodel.predict(X_test)
f1_score(y_test, pred)

0.989247311827957

![image.png](https://lms-cdn.skillfactory.ru/assets/courseware/v1/e62f9c293fb2c6ffd7e438197bf03e90/asset-v1:Skillfactory+DSMED+2023+type@asset+block/MATHML_md9_8_1.png)

![image.png](https://lms-cdn.skillfactory.ru/assets/courseware/v1/51c72308fa8d6dad07bd9a5f09a83ab6/asset-v1:Skillfactory+DSMED+2023+type@asset+block/MATHML_md9_8_2.png)

![image.png](https://lms-cdn.skillfactory.ru/assets/courseware/v1/7c28d960b938cd73e5908b9746548738/asset-v1:Skillfactory+DSMED+2023+type@asset+block/MATHML_md9_8_3.png)